# <div style="text-align: center"> Convex optimization for machine learning: Part 2</div>
#### <div style="text-align: right"> Prof. Changho Suh, TA Gyeongjo Hwang, Doyeon Kim</div>

## Implement XOR function using linear classifier

 **XOR table:**
 
 <img src="figures/XOR_table.png" style="width:270px;height:150px;">
 
 **XOR problem:** 
 
 앞서 구현한 One layer neural network로 XOR gate 구현이 불가능함

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from OL_utils import sigmoid, Logistic, plot
plt.rcParams['figure.figsize'] = (11, 8)

In [3]:
# XOR inputs & Corresponding outputs
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y_train = np.array([0, 1, 1, 0])
# Visualizing XOR gate
xlim = (-.5, 1.5)
ylim = (-.5, 1.5)
plot(X_train, Y_train, title='XOR gate', s=100, axis=True, xlim=xlim, ylim=ylim)

In [5]:
# 학습
d = Logistic(X_train, Y_train, X_train, Y_train, num_iterations=2000, learning_rate=0.1, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.693147
Cost after iteration 200: 0.693147
Cost after iteration 300: 0.693147
Cost after iteration 400: 0.693147
Cost after iteration 500: 0.693147
Cost after iteration 600: 0.693147
Cost after iteration 700: 0.693147
Cost after iteration 800: 0.693147
Cost after iteration 900: 0.693147
Cost after iteration 1000: 0.693147
Cost after iteration 1100: 0.693147
Cost after iteration 1200: 0.693147
Cost after iteration 1300: 0.693147
Cost after iteration 1400: 0.693147
Cost after iteration 1500: 0.693147
Cost after iteration 1600: 0.693147
Cost after iteration 1700: 0.693147
Cost after iteration 1800: 0.693147
Cost after iteration 1900: 0.693147
train accuracy: 50.0 %
test accuracy: 50.0 %


In [6]:
w = d['w']
b = d['b']
draw_boundary(w, b, X_train, Y_train, title='XOR gate', colormap=False, s=100, axis=True, xlim=xlim, ylim=ylim)

NameError: name 'draw_boundary' is not defined

앞 시간에 구현한 Least Square classifier, Logistic Regression은 모두 선형분류기(Linear classifier)이므로 위와 같이 Non-linear한 데이터에 대해서는 학습이 어려움

## 4 - 신경망 모델(Neural Network model)

로지스틱 회귀(Logistic Regression)은 기본적으로 선형 판별자(Linear Classifier)이기 때문에 비선형 데이터(Non-linear dataset)에 대해서는 잘 동작하지 않는다. 이를 해결하기 위해 hidden layer 1층을 추가한 신경망(Nueral Network)을 구현할 것이다. 

**Here is our model**:
<img src="figures/classification_kiank.png" style="width:600px;height:300px;">

**Mathematically**:

각각의 데이터 $x^{(i)}$에 대해:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1] (i)}\tag{1}$$ 
$$a^{[1] (i)} = ReLU(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2] (i)}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

데이터셋에 대해 손실함수(Loss function):
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Reminder**: 신경망을 구현하기 위한 Steps:
    1. 신경망 구조를 정의 (e.g., number of input units,  number of hidden units, etc). 
    2. 모델 파라미터 초기화
    3. Loop:
        - forward 전파
        - 손실 값 계산
        - 그레디언트 계산을 위한 backward 전파
        - 파라미터 업데이트(Gradient descent algorithm)


**시그모이드의 한계점**

<img src="figures/sigmoid.png" style="width:300px;height:300px;">


위 그래프에서 알 수 있듯이, x의 값이 양 끝으로 향하면 그 지역의 기울기는 매우 작아서 거의 0에 가까워진다. 즉, y의 값들이 x의 반응에 덜 반응하는 경향이 있는데, 이는 'Gradient Vanishing'이라는 문제를 야기시킨다. Backward 연산동안, 이러한 0에 가까운 기울기가 계속 곱해짐에 따라 그레디언트가 거의 0에 수렴하여 네트워크는 더이상 학습이 어려워지거나 급격하게 천천히 학습한다.

### Why ReLU?



<img src="figures/relu.png" style="width:300px;height:300px;">

ReLU는 그래프에서 보이는 것처럼 0보다 작을 때 계속 0의 값을 가지다가, 0의 값 이상일 때 선형 관계를 가지는 형태이다. 이는 0보다 큰 값에 대해서는 그래디언트가 계속 1의 값을 가지기 때문에 앞서 언급한 'Gradient Vanishing'문제는 어느정도 해결할 수 있다.

하지만 음수의 활성은 업데이트 하지 못하는 문제가 있고, 목적함수가 볼록함수(Convex)가 아니므로 'Local optimal' 지점이 존재하여 이는 학습을 더 어렵게 만들 수 있다.

In [ ]:
# ReLU 함수정의
def ReLU(z):
    return np.maximum(0, z)

### 4.1 - 신경망의 구조 정의 ####

**실습:** 다음 세개의 변수를 지정하시오:
    - n_x: 입력단(input layer)의 크기(number of units)
    - n_h: hidden layer의 크기(number of units) --> (4로 세팅) 
    - n_y: 출력단(output layer)의 크기(number of units)

**힌트:** n_x와 n_y는 X와 Y의 shape을 이용해서 구하고 n_h는 직접 손으로 작성

In [ ]:
# GRADED FUNCTION: layer_sizes

def layer_sizes(X, Y):
    """   
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    n_x = X.shape[0] # size of input layer
    n_h = 4
    n_y = Y.shape[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [ ]:
X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

### 4.2 - 모델 파라미터 초기화(Initialization) ####

**실습:** `initialize_parameters()`함수 구현

**Instructions**:
- 파라미터 shape 체크하기
- 파라미터의 각 요소는 랜덤 값으로 initialization
    - Use: `np.random.randn(a,b) * 0.01` ,(a,b)는 각 파라미터 shape.
- bias는 0으로 initialization
    - Use: `np.zeros((a,b))`

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2019)
    
    ### START CODE HERE ### (≈ 4 lines of code)
#     W1 = np.random.randn(n_h, n_x) * 0.01
#     b1 = np.zeros((n_h, 1))
#     W2 = np.random.randn(n_y, n_h) * 0.01
#     b2 = np.zeros((n_y, 1))
    W1 = np.random.randn(n_h, n_x)
    b1 = np.random.randn(n_h, 1)
    W2 = np.random.randn(1, n_h)
    b2 = np.random.randn(1, 1)
    ### END CODE HERE ###
    
    # 파라미터 shape 체크
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
n_x, n_h, n_y = initialize_parameters_test_case()

parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:

<table style="width:70%">
  <tr>
    <td>W1 :</td>
    <td> [[-0.00416758 -0.00056267]
 [-0.02136196  0.01640271]
 [-0.01793436 -0.00841747]
 [ 0.00502881 -0.01245288]] </td> 
  </tr>
  
  <tr>
    <td>b1 :</td>
    <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td> 
  </tr>
  
  <tr>
    <td>W2 :</td>
    <td> [[-0.01057952 -0.00909008  0.00551454  0.02292208]]</td> 
  </tr>
  

  <tr>
    <td>b2 :</td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 4.3 - The Loop ####

**실습:** `forward()`함수를 구현하시오

**Instructions**:
- 위의 mathematical 수식 부분 참조
- `sigmoid()` 함수는 OL_utils.py에서 일괄적으로 임포트
- `ReLU()`함수 사용
- 단계적으로 구현해야 할 것:
    1. 로지스틱 구현할때와 마찬가지로 `parameters[".."]`딕셔너리로부터 파라미터를 추출 (`initialize_parameters()`함수의 출력)
    2. forward를 구현하고 $Z^{[1]}, A^{[1]}, Z^{[2]}$ and $A^{[2]}(=\hat{Y})$를 계산 (학습데이터셋에 대한 예측(prediction)의 벡터를 나타낸다).
- backward 전파 연산에 필요한 값들은 "`cache`"에 저장. 이 `cache`는 backward()함수의 인풋으로 들어갈 것

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward(X, parameters):
    """
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2(=Yhat) (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    Z1 = np.dot(W1, X) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
X_assess, parameters = forward_propagation_test_case()

A2, cache = forward(X_assess, parameters)

# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']), np.mean(cache['A1']), np.mean(cache['Z2']), np.mean(cache['A2']))

**Expected Output**:
<table style="width:55%">
  <tr>
    <td> -0.000499755777742 -0.000496963353232 0.000438187450959 0.500109546852 </td> 
  </tr>
</table>


위에서 계산한 $A^{[2]}(=\hat{Y})$ (in the Python variable "`A2`")는 모든 데이터에 대한 확률예측값인 $a^{[2](i)}(=\hat{y}^{(i)})$의 벡터이다. 손실함수는 다음과 같이 계산한다.

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large{(} \small y^{(i)}\log\left(\hat{y}^{(i)}\right) + (1-y^{(i)})\log\left(1- \hat{y}^{(i)}\right) \large{)} \small\tag{13}$$

**실습:**

$J$ 계산을 위한`compute_cost()`함수를 구현하시오

**Instructions:**

$- \sum\limits_{i=1}^{m}  y^{(i)}\log(\hat{y}^{(i)})$ 식을 구현하기 위한 코드는 아래와 같다.
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # 루프없이 행렬연산으로만 구현가능하다
```

(바로 `np.dot()`함수를 써도 됨)

In [ ]:
# GRADED FUNCTION: compute_cost

def compute_cost(A2, Y, parameters):
    """
    Returns:
    cost -- cross-entropy cost given equation (13)
    """
    
    m = Y.shape[1] # number of example
    
    # Retrieve W1 and W2 from parameters
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
    
    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)     # 파이썬 숫자로 출력하기 위함
                                                # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
A2, Y_assess, parameters = compute_cost_test_case()

print("cost = " + str(compute_cost(A2, Y_assess, parameters)))

**Expected Output**:
<table style="width:50%">
  <tr>
    <td>cost :</td>
    <td> 0.692919893776 </td> 
  </tr>
  
</table>

forward동안 계산된 cache를 이용하여 backward propagation 연산을 구현할 수 있다.

**실습 :** `backward()` 함수를 구현하시오.

**Instructions**:

실제로 backpropagation연산 구현은 딥러닝에서 굉장히 어려운 일에 속한다. 하단 오른편 6개의 식을 참고하여 구현한다. (왼편 식의 vectorized version)

<img src="figures/grad_summary.png" style="width:600px;height:300px;">

$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (\hat{y}^{(i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * \mathbb 1(z_1^{(i)} \geq 0) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- $*$ 는 요소별 곱을 의미
- 각 그래디언트를 나타내는 notation들은 다음과 같다
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$


In [ ]:
# GRADED FUNCTION: backward_propagation

def backward(parameters, cache, X, Y):
    """
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
        
    # Retrieve also A1, Z1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = cache['A1']
    Z1 = cache['Z1']
    A2 = cache['A2']
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 8 lines of code, corresponding to 6 equations on slide above)
    dZ2= (A2 - Y) / m
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(W2.T, dZ2)
    dA1[Z1 < 0] = 0
    dZ1 = dA1
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
parameters, cache, X_assess, Y_assess = backward_propagation_test_case()

grads = backward(parameters, cache, X_assess, Y_assess)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

**Expected output**:



<table style="width:80%">
  <tr>
    <td>dW1 :</td>
    <td> [[ 0.01018708 -0.00708701]
 [ 0.00873447 -0.0060768 ]
 [-0.00530847  0.00369379]
 [-0.02206365  0.01535126]] </td> 
  </tr>
  
  <tr>
    <td>db1 :</td>
    <td>  [[-0.00069728]
 [-0.00060606]
 [ 0.000364  ]
 [ 0.00151207]] </td> 
  </tr>
  
  <tr>
    <td>dW2 :</td>
    <td> [[ 0.00363613  0.03153604  0.01162914 -0.01318316]] </td> 
  </tr>
  

  <tr>
    <td>db2 :</td>
    <td> [[ 0.06589489]] </td> 
  </tr>
  
</table>  

**실습 :** 경사하강 알고리즘(SGD)을 이용하여 업데이트 함수를 정의하시오. (위에서 계산한(dW1, db1, dW2, db2)를 이용하여 파라미터 (W1, b1, W2, b2)를 업데이트)

**General gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$  ($\alpha$는 learning rate,  $\theta$는 각 파라미터를 의미한다.

**Note**: 초기 learning rate값을 어떻게 잡느냐에 따라 학습 후 성능에 큰 영향을 미칠 수 있다.

<img src="figures/sgd.gif" style="width:400;height:400;"> <img src="figures/sgd_bad.gif" style="width:400;height:400;">

In [ ]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate=0.01):
    """    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, learning_rate=1.2)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:


<table style="width:80%">
  <tr>
    <td>W1 :</td>
    <td> [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]</td> 
  </tr>
  
  <tr>
    <td>b1 :</td>
    <td> [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]</td> 
  </tr>
  
  <tr>
    <td>W2 :</td>
    <td> [[-0.01041081 -0.04463285  0.01758031  0.04747113]] </td> 
  </tr>
  

  <tr>
    <td>b2 :</td>
    <td> [[ 0.00010457]] </td> 
  </tr>
  
</table>  

### 4.4 - 4.1, 4.2, 4.3에서 구현한 코드 NN_2L()함수로 합치기 ####

위에서 neural network를 학습하기 위한 함수들을 하나의`NN_2L()`함수로 구현
- X, Y, n_h, num_iterations 등을 인수로 받고 파라미터 딕셔너리를 리턴
- Loop statement를 이용해 num_iterations만큼 파라미터 업데이트

In [ ]:
def NN_2L(X, Y, n_h, num_iterations=10000, learning_rate=0.01, print_cost=False):
    """
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    print(n_x, n_y)
    
    # Initialize parameters, then retrieve W1, b1, W2, b2. Inputs: "n_x, n_h, n_y". Outputs = "W1, b1, W2, b2, parameters".
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    print(W1)
    print(b1)
    print(W2)
    print(b2)


    # Loop (gradient descent)
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward(X, parameters)
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, Y, parameters)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))

    return parameters

In [ ]:
X_assess, Y_assess = nn_model_test_case()

parameters = NN_2L(X_assess, Y_assess, 4, num_iterations=10000, learning_rate=1.2, print_cost=False)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:

<table style="width:90%">
  <tr>
    <td>W1 :</td>
    <td> [[-4.18494056  5.33220609]
 [-7.52989382  1.24306181]
 [-4.1929459   5.32632331]
 [ 7.52983719 -1.24309422]]</td> 
  </tr>
  
  <tr>
    <td>b1 :</td>
    <td> [[ 2.32926819]
 [ 3.79458998]
 [ 2.33002577]
 [-3.79468846]]</td> 
  </tr>
  
  <tr>
    <td>W2 :</td>
    <td> [[-6033.83672146 -6008.12980822 -6033.10095287  6008.06637269]] </td> 
  </tr>
  

  <tr>
    <td>b2 :</td>
    <td> [[-52.66607724]] </td> 
  </tr>
  
</table>  

### 4.5 데이터 클래스 예측(prediction)

**실습 :** 클래스를 예측하는 predict()함수를 구현하시오.

 - forward함수에서 출력된 값을 이용한다.

**Reminder**: predictions = $y_{prediction} = \mathbb 1 \text{{activation > 0.5}} = \begin{cases}
      1 & \text{if}\ activation > 0.5 \\
      0 & \text{otherwise}
    \end{cases}$  
    
$\hat{Y}$값을 thredhold값에 따라 prediction에 1또는 0을 넣어주고 싶다면

```X_new = (X > threshold)```

를 이용하라

In [ ]:
# GRADED FUNCTION: predict

def predict(parameters, X):
    """
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = forward(X, parameters)
    predictions = np.round(A2)
    ### END CODE HERE ###
    
    return predictions

In [ ]:
parameters, X_assess = predict_test_case()

predictions = predict(parameters, X_assess)
print("predictions mean = " + str(np.mean(predictions)))

**Expected Output**: 


<table style="width:40%">
  <tr>
    <td>predictions mean :</td>
    <td> 0.666666666667 </td> 
  </tr>
  
</table>

## Non-Linear Synthetic Data 학습

 - XOR function
 - Circle Dataset
 - Moon shaped Dataset

In [ ]:
# XOR inputs & Corresponding outputs
X_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T
Y_train = np.array([[0, 1, 1, 0]])

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
parameters = NN_2L(X_train, Y_train, n_h=2, num_iterations=10000, learning_rate=0.1, print_cost=True)

In [ ]:
draw_boundary_nn(parameters, X_train, Y_train,  title='XOR gate', s=100, axis=True, xlim=xlim, ylim=ylim, colormap=True)